# Overview

This is the analysis script for the Cog Sci 2022 proceedings write-up. 

All figures in the manuscript, along with stats reported, can be found here.

The analysis is organized with figures first and statistics at the end.

# Initialization

In [ ]:
# import libraries
import os
import math
import socket
import pymongo as pm

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_context('talk')
sns.set_style('white')

import matplotlib
from scipy import stats

In [ ]:
# Globals
proj_dir = os.path.abspath('..')
analysis_dir = os.path.join(proj_dir, 'analysis')
data_dir = os.path.join(proj_dir, 'data')
plots_dir = os.path.join(analysis_dir, 'figures')

TRIAL_CSV = 'physics_continual_learning_trialdata_e2_full.csv'
SURVEY_CSV = 'physics_continual_learning_surveydata_e2_full.csv'

In [ ]:
# Read in data
from ast import literal_eval

T = pd.read_csv(os.path.join(data_dir, TRIAL_CSV), 
                converters={'paddle_tr': literal_eval} # need this to process lists in this column
               )
S = pd.read_csv(os.path.join(data_dir, SURVEY_CSV))


# Process data

## Trial data

Add manually derived columns (doing this here to show which columns are calculated and not part of experimental logging)

In [ ]:
def cal_intervene(agent, par):
    return abs((agent - par + math.pi) % (2*math.pi) - math.pi)

def cal_final_angle(agent, par):
    if len(par) == 0: return agent
    else: return par[-1]
    
def cal_signed_error(agent, par, gt):
    sign_bot = (agent - gt + math.pi) % (2*math.pi) - math.pi
    sign_par = (par - gt + math.pi) % (2*math.pi) - math.pi
    par_sign_error = abs((par - gt + math.pi) % (2*math.pi) - math.pi) if sign_bot*sign_par > 0 else -abs((par - gt + math.pi) % (2*math.pi) - math.pi)
    return par_sign_error

def cal_normalized_signed_error(agent, par, gt):
    sign_bot = (agent - gt + math.pi) % (2*math.pi) - math.pi
    sign_par = (par - gt + math.pi) % (2*math.pi) - math.pi
    par_sign_error = sign_par/sign_bot
    return par_sign_error

In [ ]:
T['final_par'] = T.apply(lambda x: cal_final_angle(x['paddle_rho'], list(x['paddle_tr'])), axis=1)
T['intervene_dist'] = T.apply(lambda x: cal_intervene(x['paddle_rho'], x['final_par']), axis=1)
T['intervene_dist_degrees'] = T['intervene_dist'] * (180 / math.pi)

T['bot_error'] = T.apply(lambda x: cal_intervene(x['groundtruthAngle'], x['paddle_rho']), axis=1)
T['human_error'] = T.apply(lambda x: cal_intervene(x['groundtruthAngle'], x['final_par']), axis=1)
T['human_error_deg'] = T['human_error'] * (180 / math.pi)
T['bot_error_deg'] = T['bot_error'] * (180 / math.pi)

T['error_diff'] = T['bot_error'] - T['human_error']
T['normalized_signed_human_error'] = T.apply(lambda x: cal_normalized_signed_error(x['paddle_rho'], x['final_par'], x['groundtruthAngle']), axis=1)
T['signed_human_error'] = T.apply(lambda x: cal_signed_error(x['paddle_rho'], x['final_par'], x['groundtruthAngle']), axis=1)
T['signed_human_error_deg'] = T['signed_human_error'] * (180 / math.pi)


condition_lookup = {'bad': 'unreliable', 'good': 'reliable',
                    'improve': 'improving', 'worsen': 'worsening'}
T['condition_str'] = T.agent_cond.map(condition_lookup)

# Invert `trustedAgent` column to get `intervention` column
T["paddleIntervention"] = ~(T["trustedAgent"]).astype("bool")

# Add trial bins (blocks of 12 trials)
n_bins = 8
n_rounds = 96
bin_labs = [str(int(round(a * (n_rounds / n_bins), 0))) for a in range(1, n_bins + 1)]
T["trial_block"] = pd.cut(T["trialInd"], bins=8, labels=bin_labs)

# Get data for intervention trials only
intervention_trials = T[T['paddleIntervention'] == True]


## Survey data

In [ ]:
# Add relevant survey columns
S['condition_str'] = S.agent_cond.map(condition_lookup)

# add in likert responses
competence_likert = {
    'notCompetent': 0,
    'slightlyCompetent': 1,
    'moderatelyCompetent': 2,
    'veryCompetent': 3
}

S['competence_likert'] = S.agent_competence.map(competence_likert)



to_int = [
    'age',
    'intervene_rate',
    'expected_intervene_rate', 
    'physics class number'
]
S[to_int] = S[to_int].astype(int)


S = S[~S.agent_cond.isna()]
S.loc[:, 'gt_intervene_rate'] = 100 - S.gameID.map(T.groupby('gameID').trustedAgent.mean().to_dict()).copy() * 100
S['intervene_acc'] = S.gt_intervene_rate - S.intervene_rate

# Analysis

In [ ]:
# Figure styles, other generic analysis features

# source: https://htmlcolorcodes.com/color-chart/
palette = {"unreliable": "#E74C3C", # red
          "reliable": "#2980B9", # blue
          "improving": "#7D3C98", # dark purple
          "worsening": "#AF7AC5" # light purple
          }

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


**Condition summary**

Note: additional participant demographics are reported in the statistics section below

In [ ]:
# How many subjects per condition?
T.groupby(['condition_str'])['gameID'].nunique().reset_index()

## Intervention rates (Fig. 2)

In [ ]:
# Subject level intervention rate by trial block

subj_int_rate = T.groupby(
    ['gameID', 'trial_block']
).agg(
    intervention_rate = ('paddleIntervention', 'mean')
).reset_index()

# NB: this is extremely ugly please send help
conditions = T.groupby(
    ['gameID', 'trial_block']
)['condition_str'].unique().reset_index()

conditions['cond'] = np.zeros(len(conditions))
for i in range(len(conditions)):
    conditions.loc[i, 'cond'] = conditions.condition_str[i][0]

subj_int_rate['condition_str'] = conditions['cond']

subj_int_rate

In [ ]:
# Figure: proportion of interventions by trial block

plt.figure(figsize=(3,8))
fig, ax = plt.subplots()

sns.pointplot(data=subj_int_rate, 
             ax=ax,
             x="trial_block", 
             y="intervention_rate", 
             hue="condition_str",
             palette=palette
            )

ax.yaxis.grid(True)
plt.ylim(0.5, 1.0)
sns.despine(top=True, right=True)

# Figure settings for write up (modified axes and legend in Adobe Illustrator)
plt.xlabel("")
plt.ylabel("")
plt.title("")
plt.legend().remove()

# Figure settings for interpretability: toggle comment to view
# ax.set_xlabel("Trial index")
# ax.set_ylabel("Intervention rate")
# ax.legend(title="Condition",
#           loc='center left',
#           bbox_to_anchor=(1, 0.3))


plt.savefig(os.path.join(plots_dir, 'intervention_rate_subj.pdf'), dpi=300, bbox_inches='tight', transparent=True)

## Error distributions (Fig. 2)

In [ ]:
# By-subject average signed error
subj_error = T.groupby(['gameID', 'condition_str'])['signed_human_error_deg'].agg(
    subj_mean = np.mean
).reset_index()

In [ ]:
# Figure: distribution of signed errors by condition

plt.figure(figsize=(3,8))
fig, ax = plt.subplots()

sns.kdeplot(data=subj_error,
              ax=ax,
              x='subj_mean', 
              hue='condition_str',
              bw_adjust=1, # default: 1
              linewidth=4,
              palette=palette)
plt.axvline(x=0, ls='--', c='black', alpha=1)

plt.axvline(x=np.mean(subj_error.subj_mean[subj_error.condition_str=='reliable']), 
            ls='-.', c=palette['reliable'], alpha = 1, linewidth=2)
plt.axvline(x=np.mean(subj_error.subj_mean[subj_error.condition_str=='improving']), 
            ls='-.', c=palette['improving'], alpha = 1, linewidth=2)
plt.axvline(x=np.mean(subj_error.subj_mean[subj_error.condition_str=='worsening']), 
            ls='-.', c=palette['worsening'], alpha = 1, linewidth=2)
plt.axvline(x=np.mean(subj_error.subj_mean[subj_error.condition_str=='unreliable']), 
            ls='-.', c=palette['unreliable'], alpha = 1, linewidth=2)


plt.xlim(-5.5, 15.5)
ax.xaxis.grid(True)
sns.despine(top=True, right=True, left=True)

# Figure settings for write up (modified axes and legend in Adobe Illustrator)
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels([])
plt.legend().remove()

# Figure settings for interpretability: toggle comment to view
# ax.set_xlabel("Signed human error (deg.)")
# ax.legend(title="Condition",
#           labels = ["Worsening", "Reliable", "Improving", "Unreliable"], # NB: this order matters here! 
#           loc='center left',
#           bbox_to_anchor=(1, 0.3))

plt.savefig(os.path.join(plots_dir, 'error_distributions_individ_combined.pdf'), dpi=300, bbox_inches='tight', transparent=True)


## Critical trials (Fig. 3)

In [ ]:
# Isolate critical trials
crit = T.loc[T['criticalTrial'] == True, :]

In [ ]:
# Figure: average critical trial intervention *rate* by condition

plt.figure(figsize=(3,10))
fig, ax = plt.subplots()

sns.barplot(data=crit,
            ax = ax,
            x="condition_str",
            y="paddleIntervention",
            hue="condition_str",
            order=["unreliable", "improving", "worsening", "reliable"],
            alpha=0.75,
            palette=palette,
            dodge=False)

plt.axhline(y=1.0, ls='--', c='black')
sns.despine(top=True, right=True)
ax.yaxis.grid(True)
plt.ylim(0.65, 1.05)

# Figure settings for write up (modified axes and legend in Adobe Illustrator)
ax.set_xlabel("")
ax.set_xticklabels([])
ax.set_ylabel("")
ax.set_title("")
plt.legend().remove()


# Figure settings for interpretability: toggle comment to view
# ax.set_ylabel("Critical trial intervention rate")
# ax.set_xticklabels(['Unreliable', 'Improving', 'Worsening', 'Reliable'])


plt.savefig(os.path.join(plots_dir, 'critical_trial_intervention_rate.pdf'), 
            dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# Figure: average critical trial intervention *magnitude* by condition

plt.figure(figsize=(3,10))
fig, ax = plt.subplots()

sns.barplot(data=crit[crit.paddleIntervention==True],
            ax = ax,
            x="condition_str",
            y="intervene_dist_degrees",
            hue="condition_str",
            alpha=0.75,
            order=["unreliable", "improving", "worsening", "reliable"],
            palette=palette,
            dodge=False)
plt.axhline(y=16.04, c="k", ls="--")

ax.yaxis.grid(True)
plt.ylim(10, 21)
ax.set_yticks([12, 14, 16, 18, 20])
ax.set_yticklabels(["12", "", "16", "", "20"])
sns.despine(top=True, right=True)

# Figure settings for write up (modified axes and legend in Adobe Illustrator)
ax.set_xlabel("")
ax.set_xticklabels([])
ax.set_ylabel("")
ax.set_title("")
plt.legend().remove()

# Figure settings for interpretability: toggle comment to view
# ax.set_title("Critical trial interventions")
# ax.set_ylabel("Intervention magnitude (deg.)")
# ax.set_xticklabels(['Unreliable', 'Improving', 'Worsening', 'Reliable'])




plt.savefig(os.path.join(plots_dir, 'critical_trial_intervention_dist.pdf'), 
            dpi=300, bbox_inches='tight', transparent=True)

## Survey responses (Fig. 4)

In [ ]:
# Convert survey to long form, modify intervention rate

dfs = S.melt(id_vars='condition_str', 
             value_vars=['expected_intervene_rate', 'intervene_rate'], 
             value_name='intervene_rate')
dfs['intervene_rate'] = dfs['intervene_rate'] / 100 # make formatting consistent with other intervention rate plots

dfs

In [ ]:
# Figure: average reported intervention rates and expected intervention rates by condition

plt.figure(figsize=(3,8))
fig, ax = plt.subplots()

g=sns.pointplot(
    data=dfs,
    ax=ax,
    y='intervene_rate', 
    x='variable', 
    hue='condition_str',
    palette=palette,
    order=['intervene_rate', 'expected_intervene_rate'],
#     dodge = True # NB: original figure did not have this set but it's probably better...
)

ax.set_ylim(0.45, 0.95)
ax.yaxis.grid(True)
sns.despine(top=True, right=True)

# Figure settings for write up (modified axes and legend in Adobe Illustrator)
ax.set_xticklabels(["", ""])
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title("")
plt.legend().remove()

# Figure settings for interpretability: toggle comment to view
# ax.set_ylabel("Intervention rate")
# ax.set_xticklabels(["Reported (past)", "Expected (future)"])
# ax.legend(title="Condition",
#           loc='center left',
#           bbox_to_anchor=(1, 0.3))


plt.savefig(os.path.join(plots_dir, 'survey_intervention_expectations.pdf'), dpi=300)

# Statistics

Note except where indicated above, all statistical analyses are run in R through the jupyter R interface initialized below.

## Initialization

In [ ]:
%load_ext rpy2.ipython

In [ ]:
# Drop all list columns
T_clean = T.drop(['trajectory', 'paddle_tr', 'movePaddleTime', 'response'], axis=1)
df = T_clean

In [ ]:
%Rpush df
%Rpush S

In [ ]:
%%R
library(tidyverse)
library(lme4)
library(emmeans)

glimpse(df)

## Demographics

In [ ]:
%%R

# Demographics
print("AGE")
print(summary(S$age))
print(sd(S$age))

print("GENDER")
print(table(S$gender))

print("EDUCATION")
print(table(S$edu))

## Overall Performance (accuracy, RMSE)

In [ ]:
%%R

# Percent correct by subject
subj_acc = df %>%
    group_by(gameID, condition_str) %>%
    summarize(subj_mean_acc = sum(correct) / n())

print(mean(subj_acc$subj_mean_acc))
print(sd(subj_acc$subj_mean_acc))


# Percent correct by subject by trial block
subj_block_acc = df %>%
    group_by(gameID, condition_str, trial_block) %>%
    summarize(subj_mean_block_acc = sum(correct) / n())

subj_block_acc %>%
    group_by(trial_block) %>%
    summarize(
        mean_acc = mean(subj_mean_block_acc),
        sd_acc = sd(subj_mean_block_acc)
    )

In [ ]:
%%R
# subject RMSE

subj_rmse = df %>%
    group_by(gameID, condition_str) %>%
    summarize(subj_mean_rmse = sqrt(mean(human_error_deg^2)),
             bot_mean_rmse = sqrt(mean(bot_error_deg^2)))

print(mean(subj_rmse$subj_mean_rmse))
print(sd(subj_rmse$subj_mean_rmse))


## Intervention rates

In [ ]:
%%R

# NB: this takes ~10s to run

# role of condition and condition * session block interaction on intervention rates
intervention_slopes_lme = glmer(
    paddleIntervention ~ sessionBlock * condition_str + (1|gameID), 
    data = df,
    family = "binomial"
)

m1 = glmer(
    paddleIntervention ~ sessionBlock + condition_str + (1|gameID), 
    data = df,
    family = "binomial"
)

m0 = glmer(
    paddleIntervention ~ sessionBlock + (1|gameID), 
    data = df,
    family = "binomial"
)

m00 = glmer(
    paddleIntervention ~ (1|gameID), 
    data = df,
    family = "binomial"
)

print(summary(m0)) # Significant slope on block
print(anova(intervention_slopes_lme, m1, m0, m00)) # interaction significant


## Signed error

In [ ]:
%%R
# Signed error comparison

subj_signed_error = df %>%
    group_by(gameID, condition_str) %>%
    summarize(subj_mean_error = mean(signed_human_error_deg))
glimpse(subj_signed_error)


for(cond in unique(df$condition_str)) {
    print(paste0("CONDITION: ", cond))
    dat = subj_signed_error %>% filter(condition_str == cond)
    tst = t.test(x = dat$subj_mean_error, 
                 mu = 0)
    print(tst)
}

## Critical trials

In [ ]:
%%R

# PREAMBLE: Across *all trials*, is there an interaction between condition and bot error 
# when predicting intervention distance? i.e. does the amount people compensate for bot error depend on condition?
# NOTE: not including slope of trial index here because it's not clear why people intervening *more* over the course
# of the experiment should impact the magnitude of their interventions across trials when they *did* intervene.
# NOTE also: including the slope actually *improves* our model comparison

intervention_trials = df %>%
    filter(paddleIntervention == TRUE)

m00 = lmer(intervene_dist_degrees ~ condition_str + (1 | gameID),
          data = intervention_trials,
          REML = F
          )

m0 = lmer(intervene_dist_degrees ~ condition_str + bot_error_deg + (1 | gameID),
          data = intervention_trials,
          REML = F
         )

m1 = lmer(intervene_dist_degrees ~ condition_str * bot_error_deg + (1 | gameID),
          data = intervention_trials,
          REML = F
         )

print(anova(m1, m0, m00, test = 'LRT'))
print(emmeans(m1, specs = pairwise ~ condition_str)$contrasts)

In [ ]:
%%R

# Intervention rate (binary) 
# Random effects: random slope of trial number within subject, with correlated intercept

crit = df %>% filter(criticalTrial == TRUE)

m0 = glmer(paddleIntervention ~ (1 + trialInd | gameID), # previous: (1|gameID)
           data = crit,
           family = "binomial",
          )

m1 = glmer(paddleIntervention ~ condition_str + (1 + trialInd | gameID), # previous: (1|gameID) 
           data = crit,
           family = "binomial",
          )

print(anova(m1, m0, test = 'LRT'))
print(emmeans(m1, specs = pairwise ~ condition_str)$contrasts)

In [ ]:
%%R

# Intervention distance
crit_intervene = crit %>% filter(paddleIntervention == TRUE)

m0 = lmer(intervene_dist_degrees ~ (1|gameID),
          data = crit_intervene,
          REML = F
         )
m1 = lmer(intervene_dist_degrees ~ condition_str + (1|gameID),
          data = crit_intervene,
          REML = F
         )
print(anova(m1, m0, test = 'LRT'))
print(emmeans(m1, specs = pairwise ~ condition_str)$contrasts)

## Survey data

In [ ]:
%%R

# Interaction between improving and worsening

delta_conditions = S

delta_conditions_long = delta_conditions %>% 
    pivot_longer(
        cols = c('intervene_rate', 'expected_intervene_rate'),
        names_to = 'question',
        values_to = 'rating'
    )
# glimpse(delta_conditions_long)

anova_predictions = aov(rating ~ question * condition_str + Error(gameID/question), 
                        data = delta_conditions_long
                       )

print(summary(anova_predictions))


In [ ]:
%%R

# Follow-up t-tests

# Unreliable
print(
    t.test(
    delta_conditions_long$rating[delta_conditions_long$condition_str == "unreliable" & 
                             delta_conditions_long$question == "intervene_rate"],
    delta_conditions_long$rating[delta_conditions_long$condition_str == "unreliable" & 
                             delta_conditions_long$question == "expected_intervene_rate"],
    paired = T)
)


# Reliable
print(
    t.test(
    delta_conditions_long$rating[delta_conditions_long$condition_str == "reliable" & 
                             delta_conditions_long$question == "intervene_rate"],
    delta_conditions_long$rating[delta_conditions_long$condition_str == "reliable" & 
                             delta_conditions_long$question == "expected_intervene_rate"],
    paired = T)
)

# Worsening
print(
    t.test(
    delta_conditions_long$rating[delta_conditions_long$condition_str == "worsening" & 
                             delta_conditions_long$question == "intervene_rate"],
    delta_conditions_long$rating[delta_conditions_long$condition_str == "worsening" & 
                             delta_conditions_long$question == "expected_intervene_rate"],
    paired = T)
)

# Improving
print(
    t.test(
    delta_conditions_long$rating[delta_conditions_long$condition_str == "improving" & 
                             delta_conditions_long$question == "intervene_rate"],
    delta_conditions_long$rating[delta_conditions_long$condition_str == "improving" & 
                             delta_conditions_long$question == "expected_intervene_rate"],
    paired = T)
)


In [ ]:
%%R

# Relationship between predicted intervention rates and reported trust

cor.test(S$expected_intervene_rate, S$competence_likert)